# Imports

In [ ]:
# install additional packages if running on Emscripten
import platform

notebook_platform = platform.uname()
if notebook_platform.system == "Emscripten":
    import piplite

    await piplite.install(["openpyxl", "seaborn"])

In [ ]:
from enum import Enum
import re
import pathlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

sns.set_style("ticks", {"axes.grid": True})

# Data

In [ ]:
this_file = pathlib.Path().absolute()
data_dir = this_file.parent / "data"
data_dir

In [ ]:
radiation = Enum("rad_type", "proton Co60")
detector = Enum("detector", "mcp mts")

## Monte-Carlo results

In [ ]:
df_mc = pd.read_csv(data_dir / "interim" / "mc.csv")
df_mc

In [ ]:
# Reshaping the DataFrame using melt
df_melted = pd.melt(df_mc, id_vars="z_mm", var_name="LET_Type", value_name="LET")
df_melted["averaging"] = df_melted["LET_Type"].apply(lambda x: x[0])
df_melted["particles"] = df_melted["LET_Type"].apply(lambda x: x[4:])
df_melted = df_melted.drop(columns="LET_Type")
df_melted.LET /= 10
df_melted

In [ ]:
sns.relplot(
    data=df_melted, x="z_mm", y="LET", hue="averaging", col="particles", kind="line", facet_kws=dict(sharey=False)
)

In [ ]:
sns.relplot(
    data=df_melted, x="z_mm", y="LET", hue="particles", col="averaging", kind="line", facet_kws=dict(sharey=False)
)

## Detector data

In [ ]:
df_eff = pd.read_csv(data_dir / "interim" / "proton_eff.csv")
plexi_to_water = 1.16
shift = 1.2642
step_pmma = 0.00329
df_eff["z_mm"] = plexi_to_water * (shift + step_pmma * df_eff.pmma_wheel_steps)
df_eff

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(df_eff, x="pmma_wheel_steps", y="eff", hue="detector", errorbar=None, width=0.5, ax=ax)
ax.set_ylabel("Efficiency")

In [ ]:
sns.scatterplot(data=df_eff, x="z_mm", y="eff", hue="detector", style="detector")

In [ ]:
df_ratio = df_eff[df_eff.detector == detector.mcp.name].copy()
df_ratio.drop(columns=["energy_MeV", "pmma_wheel_steps", "detector"], inplace=True)
# rename eff to mcp_eff
df_ratio.rename(columns={"eff": "mcp_eff"}, inplace=True)
df_ratio = df_ratio.merge(
    df_eff[df_eff.detector == detector.mts.name][["z_mm", "eff"]], on="z_mm", suffixes=("_mcp", "_mts")
)
df_ratio.rename(columns={"eff": "mts_eff"}, inplace=True)
df_ratio["ratio"] = df_ratio.mts_eff / df_ratio.mcp_eff
df_ratio

In [ ]:
sns.scatterplot(data=df_ratio, x="z_mm", y="ratio")

In [ ]:
from scipy.interpolate import griddata


# Define function for interpolation
def interpolate_let(df, averaging, particles, z_mm):
    subset = df[(df["averaging"] == averaging) & (df["particles"] == particles)]

    if len(subset) == 0:
        raise ValueError("No matching rows found for the given averaging and particles.")

    points = subset[["z_mm"]].values
    values = subset["LET"].values

    interpolated_value = griddata(points, values, z_mm, method="linear")

    return interpolated_value  # Assuming you want a single value as the result


# Example usage
averaging_value = "d"
particles_value = "Primaries"
z_mm_value = 2  # Replace this with the desired z_mm value

interpolated_let = interpolate_let(df_melted, averaging_value, particles_value, z_mm_value)
print(f"Interpolated LET value: {interpolated_let}")

In [ ]:
interpolate_let(df_melted, averaging_value, particles_value, df_eff.z_mm.values)

In [ ]:
for av in df_melted.averaging.unique():
    for part in df_melted.particles.unique():
        df_eff[f"{av}LET{part}"] = interpolate_let(df_melted, av, part, df_eff.z_mm.values)
df_eff

In [ ]:
sns.scatterplot(data=df_eff, x="dLETProtons", y="eff", hue="detector", style="detector")

In [ ]:
for av in df_melted.averaging.unique():
    for part in df_melted.particles.unique():
        df_ratio[f"{av}LET{part}"] = interpolate_let(df_melted, av, part, df_ratio.z_mm.values)
df_ratio

In [ ]:
sns.scatterplot(data=df_ratio, x="dLETProtons", y="ratio")

In [ ]:
df_ratio

In [ ]:
# Reshaping the DataFrame using melt
df_ratio_melted = pd.melt(
    df_ratio, id_vars=["z_mm", "det_id", "mcp_eff", "mts_eff", "ratio"], var_name="LET_Type", value_name="LET"
)
df_ratio_melted["averaging"] = df_ratio_melted["LET_Type"].apply(lambda x: x[0])
df_ratio_melted["particles"] = df_ratio_melted["LET_Type"].apply(lambda x: x[4:])
df_ratio_melted = df_ratio_melted.drop(columns="LET_Type")
df_ratio_melted

In [ ]:
df_ratio_melted.averaging.unique()

In [ ]:
df_ratio_melted.particles.unique()

In [ ]:
sns.relplot(data=df_ratio_melted, x="ratio", y="LET", hue="averaging", col="particles")

In [ ]:
sns.relplot(data=df_ratio_melted, x="LET", y="ratio", hue="averaging", col="particles")

In [ ]:
sns.relplot(data=df_ratio_melted, x="LET", y="ratio", hue="particles", col="averaging")